# Tanzanian Wells

## 1. Overview

This notebook examines King County, WA dataset of houses and reviews how and what renovations add value to a house's sale price. <br>
The organization of this notebook follows the CRoss Industry Standard Process for Data Mining (CRISP-DM) is a process model that serves as the base for a data science process.

## 2. Business Understanding

Tanzania, a developing country with a population of over 57,000,000, faces significant challenges in providing clean and reliable water sources to its citizens. The country has a substantial number of existing water points, including water wells, but a considerable portion of these wells either require maintenance or have completely failed, resulting in limited access to clean water.

The objective of this project is to develop a machine learning classifier that can predict the condition of water wells in Tanzania. By analyzing various factors such as the type of pump, installation date, and other relevant attributes, we aim to categorize wells into different conditions, such as 'functional' or 'non-functional'. 

This predictive model will serve as a valuable tool for organizations and government agencies involved in water resource management and infrastructure development in Tanzania.

The target audience for this project is Non-Governmental Organizations (NGOs) focusing on improving access to clean water in Tanzania such as [WaterAid](https://www.wateraid.org/where-we-work/tanzania), [Charity Water](https://www.charitywater.org/our-projects/tanzania) or [Tanzania Water Project](https://www.tanzaniawaterproject.org/). 

**Goal**: predict whether a well is non functional.

## 3. Data Understanding

The data comes from drivendata.org, a platform which hosts data science competitions with a focus on social impact. The source of data provided by DrivenData is the Tanzanian Ministry of Water, and is stored by Taarifa. 

The actual dataset can be found [here](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/) under the 'Data download section'. 

4 files are indicated. The below files were downloaded and renamed as follows:
- Training set values: training_set_values
- Training set labels: training_set_labels
- Test set values: test_set_values

These are the files used for the main modeling and predictive analysis. 
<br>
The test set values file is the one used to measure the accuracy of the model.

In [1]:
# Importing the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
from matplotlib.patches import Rectangle
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [2]:
# Loading training_set_values dataset and saving it as df_values
df_values = pd.read_csv('data/training_set_values.csv')

In [3]:
# Inspecting df_values
df_values

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


The training set values has 59,400 rows, with 39 feature columns and 1 id column.

* `amount_tsh`: Total static head (amount water available to waterpoint)
* `date_recorded`: The date the row was entered
* `funder`: Who funded the well
* `gps_height`: Altitude of the well
* `installer`: Organization that installed the well
* `longitude`: GPS coordinate
* `latitude`: GPS coordinate
* `wpt_name`: Name of the waterpoint if there is one
* `num_private`: No description was provided for this feature
* `basin`: Geographic water basin
* `subvillage`: Geographic location
* `region`: Geographic location
* `region_code`: Geographic location (coded)
* `district_code`: Geographic location (coded)
* `lga`: Geographic location
* `ward`: Geographic location
* `population`: Population around the well
* `public_meeting`: True/False
* `recorded_by`: Group entering this row of data
* `scheme_management`: Who operates the waterpoint
* `scheme_name`: Who operates the waterpoint
* `permit`: If the waterpoint is permitted
* `construction_year`: Year the waterpoint was constructed
* `extraction_type`: The kind of extraction the waterpoint uses
* `extraction_type_group`: The kind of extraction the waterpoint uses
* `extraction_type_class`: The kind of extraction the waterpoint uses
* `management`: How the waterpoint is managed
* `management_group`: How the waterpoint is managed
* `payment`: What the water costs
* `payment_type`: What the water costs
* `water_quality`: The quality of the water
* `quality_group`: The quality of the water
* `quantity`: The quantity of water
* `quantity_group`: The quantity of water
* `source`: The source of the water
* `source_type`: The source of the water
* `source_class`: The source of the water
* `waterpoint_type`: The kind of waterpoint
* `waterpoint_type_group`: The kind of waterpoint

In [4]:
# Loading training_set_values dataset and saving it as df_labels
df_labels = pd.read_csv('data/training_set_labels.csv')

In [5]:
# Inspecting df_labels
df_labels

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
...,...,...
59395,60739,functional
59396,27263,functional
59397,37057,functional
59398,31282,functional


In [6]:
# Checking the unique values of the target column
df_labels['status_group'].unique()

array(['functional', 'non functional', 'functional needs repair'],
      dtype=object)

The training set labels has the same number of rows, and contains the:
* `id` 
* `target column`: status group

The status group can be defined as: 

1. functional: the waterpoint is operational and there are no repairs needed
2. functional needs repair: the waterpoint is operational, but needs repairs
3. non functional: the waterpoint is not operational

## 4. Data Preparation

### 4. a. Joining values and labels datasets together 

The first step of preparing the data is to merge both df_values and df_labels, as the latter contains the target value.
<br>
Both datasets are merged on the 'id' column.

In [7]:
# Merging both dataframes on the column 'id'
raw_df = df_values.merge(df_labels, on='id')

In [8]:
# Inspecting the new dataframe
raw_df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


The new joined dataframe contains the same number of rows as the previous datasets: 59,400. It has 1 id column, 1 target column: status_group, and 39 feature columns.

### 4. b. Preprocessing data

Preprocessing is an important step in data science pipeline because it transforms raw data into a suitable format for training models. It also contributes to improve model accuracy and performance by handling issues like missing values, removing unnecessary columns, scaling, and encoding categorical variables.

#### 4. b. 1. Verifying and handling missing data 

In [9]:
# Checking for null values
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

The column `scheme_name` has a high number of null values and is contains the same information as `scheme_management`: who operates the waterpoint.  
As a consequence, it will be dropped entirely.

In [10]:
# Dropping the column scheme_name
raw_df.drop(['scheme_name'], axis=1, inplace=True)

In [11]:
# Inspecting the values of columns containing null information 
columns_with_null = raw_df.columns[raw_df.isnull().any()].tolist()

columns_with_null

for column in columns_with_null:
    print(column)
    print(raw_df[column].unique())
    print()

funder
['Roman' 'Grumeti' 'Lottery Club' ... 'Dina' 'Brown' 'Samlo']

installer
['Roman' 'GRUMETI' 'World vision' ... 'Dina' 'brown' 'SELEPTA']

subvillage
['Mnyusi B' 'Nyamara' 'Majengo' ... 'Itete B' 'Maore Kati' 'Kikatanyemba']

public_meeting
[True nan False]

scheme_management
['VWC' 'Other' nan 'Private operator' 'WUG' 'Water Board' 'WUA'
 'Water authority' 'Company' 'Parastatal' 'Trust' 'SWC' 'None']

permit
[False True nan]



Other columns' null values will be replaced by 'Unknown' as they contain a relatively few missing values, and handling them as 'Unknown' could be used to predict whether a well is functional or not.  

In [12]:
# Filling null values with 'Unknown'
for column in columns_with_null:
    raw_df[column].fillna('Unknown', inplace=True)

In [13]:
# Verifying the dataset no longer contains any null value
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59400 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

#### 4. b. 2. Removing unnecessary columns

The below  columns will be removed for the following reasons:

1. Irrelevant for predictions (i.e. date the row was entered, waterpoint name)
2. Contains similar information as another column (i.e. extraction_type, water_quality) 
3. Contains information which would require additional conversion (i.e. region_code, district_code)

* `id`: the identification number assigned to the water well 
* `date_recorded`: The date the row was entered
* `longitude`: GPS coordinate
* `latitude`: GPS coordinate
* `wpt_name`: Name of the waterpoint if there is one
* `num_private`: undefined
* `subvillage`: Geographic location
* `region_code`: Geographic location (coded)
* `region`: Geographic location. There are 21 regions, while location by basin can be provided with 9 categories. Choosing less detailed categories is preferred to prevent creating a sparse dataframe 
* `district_code`: Geographic location (coded)
* `lga`: Geographic location
* `ward`: Geographic location
* `recorded_by`: Group entering this row of data
* `scheme_management`: Who operates the waterpoint
* `extraction_type`: The kind of extraction the waterpoint uses
* `extraction_type_group`: The kind of extraction the waterpoint uses
* `management_group`: How the waterpoint is managed
* `payment`: What the water costs
* `payment_type`: Frequency of payment: while it would be interesting to understand link between payment and well functionality, this feature has no link with the quality of water type and should be investigated separately
* `water_quality`: The quality of the water
* `quantity_group`: The quantity of water
* `source`: The source of the water
* `source_class`: The source of the water
* `waterpoint_type`: The kind of waterpoint
* `waterpoint_type_group`: provides similar information as source type and extraction type

In [14]:
# Storing the columns defined above into a list 
columns_to_drop = ['id', 'date_recorded','longitude','latitude','wpt_name','num_private','subvillage','region_code', 'region','district_code','lga','ward','recorded_by','scheme_management','extraction_type','extraction_type_group','management_group','payment', 'payment_type', 'water_quality','quantity_group','source','source_class','waterpoint_type', 'waterpoint_type_group']

In [15]:
# Dropping the columns from the dataframe and creating a new one
df = raw_df.drop(columns_to_drop, axis=1)

In [16]:
# Inspecting the new df
df

,amount_tsh,funder,gps_height,installer,basin,population,public_meeting,permit,construction_year,extraction_type_class,management,quality_group,quantity,source_type,status_group
0,6000.0,Roman,1390,Roman,Lake Nyasa,109,True,False,1999,gravity,vwc,good,enough,spring,functional
1,0.0,Grumeti,1399,GRUMETI,Lake Victoria,280,Unknown,True,2010,gravity,wug,good,insufficient,rainwater harvesting,functional
2,25.0,Lottery Club,686,World vision,Pangani,250,True,True,2009,gravity,vwc,good,enough,dam,functional
3,0.0,Unicef,263,UNICEF,Ruvuma / Southern Coast,58,True,True,1986,submersible,vwc,good,dry,borehole,non functional
4,0.0,Action In A,0,Artisan,Lake Victoria,0,True,True,0,gravity,other,good,seasonal,rainwater harvesting,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,Germany Republi,1210,CES,Pangani,125,True,True,1999,gravity,water board,good,enough,spring,functional
59396,4700.0,Cefa-njombe,1212,Cefa,Rufiji,56,True,True,1996,gravity,vwc,good,enough,river/lake,functional
59397,0.0,Unknown,0,Unknown,Rufiji,0,True,False,0,handpump,vwc,fluoride,enough,borehole,functional
59398,0.0,Malec,0,Musa,Rufiji,0,True,True,0,handpump,vwc,good,insufficient,shallow well,functional


In [17]:
# Inspecting the new df's info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   funder                 59400 non-null  object 
 2   gps_height             59400 non-null  int64  
 3   installer              59400 non-null  object 
 4   basin                  59400 non-null  object 
 5   population             59400 non-null  int64  
 6   public_meeting         59400 non-null  object 
 7   permit                 59400 non-null  object 
 8   construction_year      59400 non-null  int64  
 9   extraction_type_class  59400 non-null  object 
 10  management             59400 non-null  object 
 11  quality_group          59400 non-null  object 
 12  quantity               59400 non-null  object 
 13  source_type            59400 non-null  object 
 14  status_group           59400 non-null  object 
dtypes:

The new dataframe still has 59,400 rows, but now contains 17 feature columns and 1 target column. 
14 of the features, including the target variable is a categorical data, so they will be one-hot encoded in the next section.

#### 4. b. 3. Transforming the classification into a binary one

The target column contains 3 categories. The dataset is highly imbalanced, and the status 'functional needs repair' only represents 7% of the rows. 


A well that is functional but needs repair can be considered non-functional because it does not reliably provide safe and consistent access to water, which is the primary function of a well. <br> 
As a consequence, all 'functional needs repair' statuses will be replaced by 'non functional,'

Transforming the classification from a ternary to a binary one will then address the imbalanced dataset. 

In [18]:
# Inspecting the values inside the column status_group
print(df['status_group'].unique())

['functional' 'non functional' 'functional needs repair']


In [19]:
# Verifying for data imbalance
print('Functional counts')
print(df['status_group'].value_counts())
print()
print()
print('Functional counts')
print(df['status_group'].value_counts(normalize=True))

Functional counts
functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64


Functional counts
functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64


The dataset is highly imbalanced, and the status 'functional needs repair' only represents 7% of the rows. 


A well that is functional but needs repair can be considered non-functional because it does not reliably provide safe and consistent access to water, which is the primary function of a well. <br> 
As a consequence, all 'functional needs repair' statuses will be replaced by 'non functional,'

Transforming the classification from a ternary to a binary one will then address the imbalanced dataset. 

In [20]:
# Replacing 'functional needs repair' by 'non functional'
df['status_group'] = df['status_group'].replace('functional needs repair', 
                                                'non functional')


In [21]:
# Verifying the replacement was correctly applied
print('Raw counts')
print(df['status_group'].value_counts())
print()
print()
print('Percentages')
print(df['status_group'].value_counts(normalize=True))

Raw counts
functional        32259
non functional    27141
Name: status_group, dtype: int64


Percentages
functional        0.543081
non functional    0.456919
Name: status_group, dtype: float64


If we had a model that *always* said  that the well was non functional, we would get an accuracy score of 0.456919, i.e. 45.7% accuracy.
<br> 
This is because bout 45.7% of all wells are currently non functional. 

#### 4. b. 4. Converting other binary columns

Some categorical features are binary: true or false, so they will be replaced byL
* 0 if false
* 1 if true
<br>Some of the data contains 'unknown' data. If unknown, it will be considered false. 

The target column is not technically true or false but is binary as well, so it will be converted as the following:
* functional: 1 
* non functional: 0 

In [22]:
# Storing binary columns into a new dataframe
binary_columns = ['public_meeting', 'permit', 'status_group']

In [23]:
# Converting public_meeting, permit and status_group to binary encoding
for column in binary_columns:
    print(column, df[column].unique())
    df[column] = df[column].map({
        False: 0,
        True: 1,
        'Unknown': 0,
        'non functional': 0,
#         'functional needs repair': 0,
        'functional': 1

    }) 
    print(column, df[column].unique())


public_meeting [True 'Unknown' False]
public_meeting [1 0]
permit [False True 'Unknown']
permit [0 1]
status_group ['functional' 'non functional']
status_group [1 0]


In [24]:
# Verifying the new data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   funder                 59400 non-null  object 
 2   gps_height             59400 non-null  int64  
 3   installer              59400 non-null  object 
 4   basin                  59400 non-null  object 
 5   population             59400 non-null  int64  
 6   public_meeting         59400 non-null  int64  
 7   permit                 59400 non-null  int64  
 8   construction_year      59400 non-null  int64  
 9   extraction_type_class  59400 non-null  object 
 10  management             59400 non-null  object 
 11  quality_group          59400 non-null  object 
 12  quantity               59400 non-null  object 
 13  source_type            59400 non-null  object 
 14  status_group           59400 non-null  int64  
dtypes:

There are now 11 categorical columms. 

#### 4. b. 5. Categorizing Values With Too Many Details

##### First step: Counting Values in Categorical Variables

Some categorical variables such as funder or installer cannot be one-hot encoded directly, as they contain too many distinct values. Further data transformation is required.

In [25]:
# Creating the dataframe categoricals to handle the categorical columns
categoricals = df.select_dtypes(include=[object])
categoricals

,funder,installer,basin,extraction_type_class,management,quality_group,quantity,source_type
0,Roman,Roman,Lake Nyasa,gravity,vwc,good,enough,spring
1,Grumeti,GRUMETI,Lake Victoria,gravity,wug,good,insufficient,rainwater harvesting
2,Lottery Club,World vision,Pangani,gravity,vwc,good,enough,dam
3,Unicef,UNICEF,Ruvuma / Southern Coast,submersible,vwc,good,dry,borehole
4,Action In A,Artisan,Lake Victoria,gravity,other,good,seasonal,rainwater harvesting
...,...,...,...,...,...,...,...,...
59395,Germany Republi,CES,Pangani,gravity,water board,good,enough,spring
59396,Cefa-njombe,Cefa,Rufiji,gravity,vwc,good,enough,river/lake
59397,Unknown,Unknown,Rufiji,handpump,vwc,fluoride,enough,borehole
59398,Malec,Musa,Rufiji,handpump,vwc,good,insufficient,shallow well


In [26]:
# Storing categorical columns to a list
categorical_columns = categoricals.columns.tolist()
# Creating an empty dictionary to store value counts for each column
value_counts_dict = {}

# Iterating through each categorical column and calculating value counts
for column in categorical_columns:
    value_counts = categoricals[column].value_counts()
    value_counts_dict[column]=value_counts

In [27]:
# Reviewing columns with the highest number of categories within each of them 
value_counts_dict

{'funder': Government Of Tanzania    9084
 Unknown                   3639
 Danida                    3114
 Hesawa                    2202
 Rwssp                     1374
                           ... 
 Muwasa                       1
 Msigw                        1
 Rc Mofu                      1
 Overland High School         1
 Samlo                        1
 Name: funder, Length: 1897, dtype: int64,
 'installer': DWE           17402
 Unknown        3658
 Government     1825
 RWE            1206
 Commu          1060
               ...  
 EWE               1
 SCHOO             1
 Got               1
 Fabia             1
 SELEPTA           1
 Name: installer, Length: 2145, dtype: int64,
 'basin': Lake Victoria              10248
 Pangani                     8940
 Rufiji                      7976
 Internal                    7785
 Lake Tanganyika             6432
 Wami / Ruvu                 5987
 Lake Nyasa                  5085
 Ruvuma / Southern Coast     4493
 Lake Rukwa             

Funder and Installer are the two columns with the highest number of categories and seem to have very similar results. We will first focus on funders. 


##### Second step: Categorizing Columns Requiring It

<u>Funders</u> like any column, contains 59,400 rows. It is formed by 1,897 unique values. In order to organize them into similar categories, a research of each funder was done to identify if the organization was considered:
1. **Bilateral**: the government from another country funded the water well
2. **Government**: the government of Tanzania, or a programme funded by the government, or local, governmental agencies funded the water well  
3. **NPO_NGO**: the water well is funded by a non-profit organization or a non-governmental organization
4. **Private**: the fund comes from a private source: either individual or a company 
5. **Religious**: a religious organization funded the well
6. **Unknown & Minor**: the funder was not or could not be identified, or had funded less than 150 projects

The research was divided into two categories: 
- those which had funded at 150 projects were researched individually
- the others were categorized as minor funders

In a normal distribution, data outside the 75th percentile would be considered outliers. The goal of the above detailed classification was to get as close to the 75th percentile as possible. <br>
The column contains 59,400 rows, but currently has 3,658 funders identified as 'Unknown', leavingg 55,742 rows of funders to be categorized. Categorizing data up to the 75th percentile would classify over 41,806 rows. 


By setting the limit on funders who had paid for the wells to 150, this allowed to identify and categorize 43,177 funders, meeting our 75th percentile objective. 

In [28]:
categoricals['funder'].value_counts()

Government Of Tanzania    9084
Unknown                   3639
Danida                    3114
Hesawa                    2202
Rwssp                     1374
                          ... 
Muwasa                       1
Msigw                        1
Rc Mofu                      1
Overland High School         1
Samlo                        1
Name: funder, Length: 1897, dtype: int64

In [29]:
# Printing all rows for categorization but commented out for the rest of the code dfor better memory use
# pd.set_option('display.max_rows', None)

In [30]:
# Identifiying funders that funded up 150 water wells 
categoricals[['funder']].groupby('funder').filter(lambda x: len(x) <= 150).value_counts()

funder                       
Mkinga Distric Coun              150
Lvia                             147
Concern World Wide               145
Unhcr                            137
No                               134
                                ... 
Makori                             1
Makonder                           1
Makondakonde Water Population      1
Makona                             1
Zingibali Secondary                1
Length: 1837, dtype: int64

In [31]:
# Counting funders that funded over 150 water wells 
categoricals[['funder']].groupby('funder').filter(lambda x: len(x) > 150).value_counts()

funder                        
Government Of Tanzania            9084
Unknown                           3639
Danida                            3114
Hesawa                            2202
Rwssp                             1374
World Bank                        1349
Kkkt                              1287
World Vision                      1246
Unicef                            1057
Tasaf                              877
District Council                   843
Dhv                                829
Private Individual                 826
Dwsp                               811
0                                  777
Norad                              765
Germany Republi                    610
Tcrs                               602
Ministry Of Water                  590
Water                              583
Dwe                                484
Netherlands                        470
Hifab                              450
Adb                                448
Lga                              

In [32]:
# Ensuring enough data is categorized by counting how much would represent classiying funders that funded at least 150 wells
categoricals[['funder']].groupby('funder').filter(lambda x: len(x) > 150).value_counts().sum()

43177

A copy of the column `funder` will be created `funder_organization` to then replace each of the categories with the ones defined above.

In [33]:
# Creating column funders_organization with values from funders 
categoricals['funder_organization'] = categoricals['funder']
categoricals['funder_organization']

0                  Roman
1                Grumeti
2           Lottery Club
3                 Unicef
4            Action In A
              ...       
59395    Germany Republi
59396        Cefa-njombe
59397            Unknown
59398              Malec
59399         World Bank
Name: funder_organization, Length: 59400, dtype: object

In [34]:
# Verifying that the copy of the column was correctly done
assert (categoricals['funder_organization'] == categoricals['funder']).all(), "Columns are not equal."

# If the assertion passes, it will not raise an error.
print("Columns are equal.")

Columns are equal.


In [35]:
# Storing each of the identified funder into the corresponding list to then replace them
bilateral = ["Danida","Hesawa","Norad","Germany Republi","Netherlands","Rudep","Nethalan","World Bank","W.B"]
government = ["Government Of Tanzania","Rwssp","District Council","Dwsp","Water","Dwe","Lga","Private","Jaica","Rural Water Supply And Sanitat","Jica","Wsdp","Rc"]
NPO_NGO = ["World Vision","Unicef","Tasaf","Ministry Of Water","Amref","Oxfam","Wateraid","Mission","Shipo","Ded","Plan Int","Oxfarm","Oikos E.Afrika"]
private = ["Finw","Kiliwater","Dhv","Hifab","Adb","Fini Water","Isf","Ces(gmbh)","Fw","Ces (gmbh)","Private Individual","Lawatefuka Water Supply","Magadini-makiwaru Water"]
religious = ["Kkkt","Tcrs","Rc Church","Adra","Dmdd","Kkkt_makwale","Wvt","Roman"]
unknown = ["Unknown","0","Dh","Go", "Not Known "]

# Storing into a list the other funders: those that funded less than 150 wells
unknown_n_minor = categoricals['funder_organization'].value_counts()[categoricals['funder_organization'].value_counts() <= 150].index.tolist()



In [36]:
# Replacing each list of funders by their assigned category
categoricals['funder_organization'] = categoricals['funder_organization'].replace(bilateral, 'bilateral')
categoricals['funder_organization'] = categoricals['funder_organization'].replace(government, 'government')
categoricals['funder_organization'] = categoricals['funder_organization'].replace(NPO_NGO, 'NPO_NGO')
categoricals['funder_organization'] = categoricals['funder_organization'].replace(private, 'private')
categoricals['funder_organization'] = categoricals['funder_organization'].replace(religious, 'religious')
categoricals['funder_organization'] = categoricals['funder_organization'].replace(unknown, 'unknown_n_minor')
categoricals['funder_organization'] = categoricals['funder_organization'].replace(unknown_n_minor, 'unknown_n_minor')



In [37]:
# Verifying all strings were correctly replaced
categoricals['funder_organization'].unique()

array(['religious', 'unknown_n_minor', 'NPO_NGO', 'government',
       'bilateral', 'private'], dtype=object)

<u>Installer</u> 


<u>Installer</u> also contains 59,400 rows. The 2,145 identifiers were summarized into 8 categories, similar to funders - as follows:
1. **Bilateral**: the government from another country installed the water well
2. **Community**: installed by citizens, or the community
3. **Government**: the government of Tanzania, or a programme organized by the government, or local, governmental agencies installed the water well  
4. **NPO_NGO**: the water well was installed by a non-profit organization or a non-governmental organization
5. **Private**: the installation was done by a private source: either individual or a company 
6. **Religious**: a religious organization installed the well
7. **Unknown and minor**: the installation was not or could not be identified, or has installed less than 100 wells

The research was divided into two categories: 
- those which had installed at least 100 projects were researched individually
- the others were categorized as minor installers

Categorizing the above allowed to identify 45,658 - or 76.9%, which meets the 75th percentile goal. <br>

In [38]:
# Verified if Funder was the same as Installer, checking by chunks of 20 

# Defining the chunk size
chunk_size = 20

# Storing the total number of rows in the dataframe
total_rows = len(categoricals)

# Iterating through the DataFrame in chunks
for i in range(0, total_rows, chunk_size) :
    start_index = i
    end_index = min(i + chunk_size, total_rows)
    
    # Review the chunk of data in the 'column'
    chunk = categoricals[['funder', 'installer']][start_index:end_index]
    
    # Process the chunk (e.g., print, analyze, etc.)
    print(chunk)

                     funder             installer
0                     Roman                 Roman
1                   Grumeti               GRUMETI
2              Lottery Club          World vision
3                    Unicef                UNICEF
4               Action In A               Artisan
5       Mkinga Distric Coun                   DWE
6                      Dwsp                  DWSP
7                     Rwssp                   DWE
8                  Wateraid             Water Aid
9               Isingiro Ho               Artisan
10                  Private               Private
11                   Danida                DANIDA
12             World Vision          World vision
13  Lawatefuka Water Supply  Lawatefuka water sup
14                    Biore                WEDECO
15                    Rudep                   DWE
16                   Unicef                   DWE
17                   Unicef                   DWE
18                   Hesawa                   DWE


                      funder                  installer
3760  Government Of Tanzania                        RWE
3761                   Kirde                        DWE
3762            Singida Yetu               SINGIDA YETU
3763                 Unknown                    Unknown
3764         Germany Republi                        CES
3765                  Sabemo                  Chamavita
3766                     Vwc                        VWC
3767                   Rwssp                        DWE
3768  Government Of Tanzania                      Centr
3769             Safari Roya                 Government
3770            Village Govt               Village Govt
3771                     Dhv                        DWE
3772  Government Of Tanzania                        DWE
3773  Government Of Tanzania  District Water Department
3774                 Private                      Priva
3775                 Private                      Priva
3776                 Unknown                    

                      funder        installer
7880                    Tcrs             TCRS
7881  Government Of Tanzania            Gover
7882                     Isf              ISF
7883              Said Omari       Said Omari
7884                    Tcrs             TCRS
7885     Kibaha Town Council              DWE
7886                 Concern            TWESA
7887  Government Of Tanzania              DWE
7888         Water Aid /sema  Water Aid /sema
7889                       0                0
7890                    Tcrs             TCRS
7891                Miziriol         Miziriol
7892                     Dhv              DWE
7893               Adp Mombo              DWE
7894                     Adp           WEDECO
7895                      No             NORA
7896  Government Of Tanzania            Centr
7897                  Kuwait    AFRICA MUSLIM
7898  Government Of Tanzania              DWE
7899                  Hesawa           HESAWA
                       funder     

                       funder         installer
11740                   Unhcr              TCRS
11741                  Hesawa            HESAWA
11742                   Hifab            Hesawa
11743                 Msikiti             Msiki
11744       Ministry Of Water     Idara ya maji
11745                       0                 0
11746                  Tuwasa            TUWASA
11747                   Mdrdp             MDRDP
11748     Mkinga Distric Coun               DWE
11749  Government Of Tanzania               RWE
11750  Government Of Tanzania  District council
11751                    Dwsp               DWE
11752  Government Of Tanzania               DWE
11753                 Unknown           Unknown
11754                     Wvt               WVT
11755                 Private           Private
11756                   Rwssp            WEDECO
11757                    Cefa              CEFA
11758                  Danida            DANIDA
11759                     Twe           

                         funder         installer
15840                       Wua                WU
15841          District Council  District Council
15842    Government Of Tanzania               DWE
15843           Germany Republi               CES
15844                     Rudep               DWE
15845                Fini Water        Fini Water
15846                    Danida               DED
15847                       W.B               DWE
15848           Water Aid /sema   Water Aid /sema
15849                    Danida             DANID
15850  African Development Bank           DAWASCO
15851           Village Council   VILLAGE COUNCIL
15852                      Kkkt              KKKT
15853                     Roman             Roman
15854                     Rwssp            WEDECO
15855                       Twe               TWE
15856    Government Of Tanzania               DWE
15857                      Brdp           RUNDAGA
15858                      Tcrs              TCRS


...

                        funder             installer
35680  Magadini-makiwaru Water  Magadini-Makiwaru wa
35681                   Danida                   DWE
35682                    Norad               RWE/DWE
35683                   Danida                DANIDA
35684                      Dwe                   DWE
35685       Private Individual                    AD
35686                       St                    St
35687   Government Of Tanzania                 Gover
35688                  Unknown               Unknown
35689                      Adb     JANDU PLUMBER  CO
35690                   Danida                 DANID
35691                       Md                    DW
35692                      Snv                   DWE
35693                     Dwsp                   DWE
35694                   Danida                DANIDA
35695                    Ridep                 RIDEP
35696   Government Of Tanzania    Central government
35697   Government Of Tanzania                

                       funder         installer
39520                    Kkkt              KKKT
39521                   Roman             Commu
39522  Government Of Tanzania             TWESA
39523      Private Individual               DWE
39524                  Hesawa            HESAWA
39525                 Marafip               DWE
39526                Irish Ai               DWE
39527  Government Of Tanzania        Government
39528  Government Of Tanzania            Distri
39529                   Jaica             Jaica
39530                      Rc                RC
39531                     Hsw               HSW
39532                   Rwssp               DWE
39533                    Undp               DWE
39534  Government Of Tanzania             Centr
39535                    Jica              JICA
39536                   Jaica             JAICA
39537    Lotary International  District council
39538                 Concern           CONCERN
39539                   Roman           

                       funder                  installer
43240  Government Of Tanzania  District Water Department
43241                       0                          0
43242  Government Of Tanzania                        DWE
43243            World Vision                        DWE
43244                      Ir                         Ir
43245                     Dhv                        DWE
43246                    Cmsr                      GOVER
43247                     Dwe                        DWE
43248                 Unknown                    Unknown
43249                  Hesawa                        DWE
43250  Government Of Tanzania                        DWE
43251     Kibaha Town Council                        DWE
43252         Anglican Church                      Angli
43253                      Ir                         Ir
43254                    Lvia                      GOVER
43255  Government Of Tanzania                        DWE
43256                   Hifab  

                           funder                   installer
47440                        Jica                         DWE
47441      Government Of Tanzania                         DWE
47442                          Ki                          DW
47443           Ministry Of Water                         DWE
47444                           0                           0
47445                     Unknown                     Unknown
47446                        Kadp                        KADP
47447                           0                           0
47448                         Dhv                         DWE
47449      Government Of Tanzania            District council
47450                       Water                        Gove
47451  Living Water International  Living water international
47452                      Unicef                         DWE
47453                       Rwssp                         DWE
47454          Private Individual                          Wa
47455   

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [39]:
# Identifiying installers that funded at least 150 water wells 
categoricals[['installer']].groupby('installer').filter(lambda x: len(x)  >= 100).value_counts()

installer                    
DWE                              17402
Unknown                           3658
Government                        1825
RWE                               1206
Commu                             1060
                                 ...  
WATER AID                          103
FINI WATER                         103
Halmashauri ya wilaya sikonge      102
Wizara ya maji                     100
HSW                                100
Length: 85, dtype: int64

In [40]:
# Ensuring enough data is categorized by counting how much would represent classiying funders that funded at least 150 wells
categoricals[['installer']].groupby('installer').filter(lambda x: len(x) > 100).value_counts().sum()

45658

A copy of the column `installer` will be created `installer_organization` to then replace each of the categories with the ones defined above.

In [41]:
# Creating column funders_organization with values from funders 
categoricals['installer_organization'] = categoricals['installer']
categoricals['installer_organization']

0               Roman
1             GRUMETI
2        World vision
3              UNICEF
4             Artisan
             ...     
59395             CES
59396            Cefa
59397         Unknown
59398            Musa
59399           World
Name: installer_organization, Length: 59400, dtype: object

In [42]:
# Verifying that the copy of the column was correctly done
assert (categoricals['installer_organization'] == categoricals['installer']).all(), "Columns are not equal."

# If the assertion passes, it will not raise an error.
print("Columns are equal.")

Columns are equal.


In [43]:
# Storing each of the identified funder into the corresponding list to then replace them
bilateral_inst = ["DANIDA", "Hesawa", "NORAD", "Norad", "Kuwait", "JAICA", "HE"]
community_inst = ["Commu", "Community", "Villagers", "wananchi"]
government_inst = ["DWE", "Government", "Central government", "District Council", "LGA", "District council", "Gover","Idara ya maji","Gove","Sengerema Water Department","DWSP","Distri","Centr","Handeni Trunk Main(","RWSSP", "Central govt", "GOVER", "JICA", "Central Government","MUWSA","Halmashauri ya wilaya sikonge","HSW","Wizara ya maji"]
NPO_NGO_inst = ["World vision","TASAF","AMREF","TWESA","ACRA","World Vision","SEMA","OXFAM","UNICEF","WVT","SHIPO","Mission","World","Amref","TWE","DED","WATER AID"]
private_inst = ["RWE", "CES", "WEDECO", "WU", "DW", "Da", "Kiliwater", "FinW", "DH", "Lawatefuka water sup", "Magadini-Makiwaru wa", "FW", "MWE", "Fini Water", "Private", "Artisan", "ISF", "Fini water", "DDCA", "Tardo", "Consulting Engineer", "FINI WATER"]
religious_inst = ["KKKT","TCRS","DANID","HESAWA","Dmdd","RC","KKKT _ Konde and DWE","RC CHURCH"]
unknown_inst = ["Unknown","0","Is","Ir"]

# Storing into a list the other funders: those that funded less than 150 wells
minor_installers = categoricals['installer_organization'].value_counts()[categoricals['installer_organization'].value_counts() <= 100].index.tolist()



In [44]:
bilateral_inst

['DANIDA', 'Hesawa', 'NORAD', 'Norad', 'Kuwait', 'JAICA', 'HE']

In [45]:
# Replacing each list of installer by their assigned category
categoricals['installer_organization'] = categoricals['installer_organization'].replace(bilateral_inst, 'bilateral')
categoricals['installer_organization'] = categoricals['installer_organization'].replace(community_inst, 'community')
categoricals['installer_organization'] = categoricals['installer_organization'].replace(government_inst, 'government')
categoricals['installer_organization'] = categoricals['installer_organization'].replace(NPO_NGO_inst, 'NPO_NGO')
categoricals['installer_organization'] = categoricals['installer_organization'].replace(private_inst, 'private')
categoricals['installer_organization'] = categoricals['installer_organization'].replace(religious_inst, 'religious')
categoricals['installer_organization'] = categoricals['installer_organization'].replace(unknown_inst, 'unknown_n_minor')
categoricals['installer_organization'] = categoricals['installer_organization'].replace(minor_installers, 'unknown_n_minor')



In [46]:
# Verifying all strings were correctly replaced
len(categoricals['installer_organization'].unique())
categoricals['installer_organization'].unique()

array(['unknown_n_minor', 'NPO_NGO', 'government', 'bilateral',
       'community', 'religious'], dtype=object)

In [47]:
categoricals = categoricals.drop(['funder', 'installer'], axis=1)

<u>Management</u> 


External research was made to understand some of the terms, including [management definitions](https://washmatters.wateraid.org/publications/private-operation-in-the-rural-water-supply-in-central-tanzania-quick-fixes-and-slow) 


<u>Management</u>

* vwc: Village Water Committee: regulatory body with elected membership. They are the default management framework
* wug: Water User Group: sub-village level management groups responsible for operation and maintenance of the water point.
* water board: legal entity, autonomous from village government
* wua: Water User Association: A legal entity autonomous from village government that oversees the management of an entire extraction system.
* private operator: A privately funded extension to a water system
* parastatal: a company or organization which is owned by a country's government and often has some political power
* water authority: 
* other: 
* company: A legal entity sometimes under contract
* other - school: The water points are public but managed fully by the school
* trust: legal entity, autonomous from village government 
* unknown


Following the research on these categories, some of them can be grouped further together. 

* vwc and wug = Village Water Committee and Water User Group both refer to a village or sub-village management group 
* water board and trust = are both defined as a legal entity, autonomous from village government
* parastatal and water authority = eventually refer to organizations owned by the country's government
* other and other-school = both contain management types that cannot be defined as the above classifications
* private operator and company = both a privately funded extension to a water system


In [48]:
# Grouping the above categories together 

# vwc and wug
categoricals['management'] = categoricals['management'].replace('vwc', 'village_committee')
categoricals['management'] = categoricals['management'].replace('wug', 'village_committee')

# water board and trust
categoricals['management'] = categoricals['management'].replace('water board', 'board_n_trust')
categoricals['management'] = categoricals['management'].replace('trust', 'board_n_trust')

# water authority and parastatal
categoricals['management'] = categoricals['management'].replace('water authority', 'government')
categoricals['management'] = categoricals['management'].replace('parastatal', 'government')

# other and other school
categoricals['management'] = categoricals['management'].replace('other - school', 'other')

# private operator and company
categoricals['management'] = categoricals['management'].replace('company', 'private operator')

In [49]:
categoricals['management'].unique()

array(['village_committee', 'other', 'private operator', 'board_n_trust',
       'wua', 'government', 'unknown'], dtype=object)

In [50]:
len(categoricals['management'].unique())

7

##### Third step: Further Columns Dropping: those with Similar Information

Now that both columns were categorized, they appear to show similar groups. In order to optimize the predictions, only one of the two columns should be kept. <br>
While both funders and installers are important, when it comes to determining the functionality of a water well, the expertise and competence of the installers are usually the key factors. 
<br> The `installer_organization` column will be kept 

In [51]:
categoricals = categoricals.drop(['funder_organization'], axis=1)

In [52]:
categoricals.columns

Index(['basin', 'extraction_type_class', 'management', 'quality_group',
       'quantity', 'source_type', 'installer_organization'],
      dtype='object')

#### 4. b. 5. Encoding Categorical Variables

This step enables non-numeric data to be processed and analyzed, facilitating meaningful pattern recognition and model training.
It uses the one-hot encoding method, which converts each category as a binary vector: 1 indicating the presence of the category, 0 indicating its absence.  

In [53]:
# One-hot encoding the categorical columns
one_hot_df = pd.get_dummies(categoricals)
one_hot_df.columns

Index(['basin_Internal', 'basin_Lake Nyasa', 'basin_Lake Rukwa',
       'basin_Lake Tanganyika', 'basin_Lake Victoria', 'basin_Pangani',
       'basin_Rufiji', 'basin_Ruvuma / Southern Coast', 'basin_Wami / Ruvu',
       'extraction_type_class_gravity', 'extraction_type_class_handpump',
       'extraction_type_class_motorpump', 'extraction_type_class_other',
       'extraction_type_class_rope pump', 'extraction_type_class_submersible',
       'extraction_type_class_wind-powered', 'management_board_n_trust',
       'management_government', 'management_other',
       'management_private operator', 'management_unknown',
       'management_village_committee', 'management_wua',
       'quality_group_colored', 'quality_group_fluoride', 'quality_group_good',
       'quality_group_milky', 'quality_group_salty', 'quality_group_unknown',
       'quantity_dry', 'quantity_enough', 'quantity_insufficient',
       'quantity_seasonal', 'quantity_unknown', 'source_type_borehole',
       'source_type_d

In [54]:
# Verifying how many columns this provided
len(one_hot_df.columns)

47

#### 4. b. 6. Grouping Numeric and Encoded Categorical Variables

Now that we have our encoded categorical variables, the rest of the numeric columns dataframe will be grouped to this new one_hot_df to have a useable dataframe for our models. 

In [55]:
# Creating a numeric df
numeric_df = df.select_dtypes(include=['number'])

In [56]:
# Verifying its columns
numeric_df.columns

Index(['amount_tsh', 'gps_height', 'population', 'public_meeting', 'permit',
       'construction_year', 'status_group'],
      dtype='object')

In [57]:
# Now grouping both numeric and encoded categorical dataframes
combined_df = pd.concat([numeric_df, one_hot_df], axis=1)

## 5. Modeling

----------------- Notes ----------------- 

What modeling techniques should we apply?

Begin with a basic model, evaluate it, and then provide justification for and proceed to a new model. 



Be sure to explore:

1. Model features and preprocessing approaches
2. Different kinds of models (logistic regression, k-nearest neighbors, decision trees, etc.)
3. Different model hyperparameters

At minimum you must build three models:

* A simple, interpretable baseline model (logistic regression or single decision tree)
* A more-complex model (e.g. random forest)
* A version of either the simple model or more-complex model with tuned hyperparameters

----------------- Notes ----------------- 

### 5. a. Logistic Regression

#### 5. a. 1. Performing a Train-Test Split

In [58]:
# Splitting df into X and y
X = combined_df.drop('status_group', axis=1)
y = combined_df['status_group']

The dataset is being divided into two separate subsets: a training set, and a testing (or validation) set. The validation set will allow to assess the performance of the model. 

Two parameters are assigned when dividing the dataset:
* random_state=42 
   - setting a random seed of 42 ensures that the data split is reproducible
* stratify=y 
   - stratified sampling ensures the class distribution is maintained in both sets to address potential class imbalance issues

In [59]:
# Performing train-test split with random_state=42 and stratify=y 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

Because stratify=y was applied, the percentages of non functional water wells in the train and test target should be similar. 

In [60]:
# Inspecting the percentages of non functional water wells in train and test targets: 
print("Train percent of non functional wells:", y_train.value_counts(normalize=True)[1])
print("Test percent of non functional wells:", y_test.value_counts(normalize=True)[1])

Train percent of non functional wells: 0.5430751964085297
Test percent of non functional wells: 0.5430976430976431


#### 5. a. 2. Building and Evaluating a Baseline Model

We will begin by building  a vanilla logistic regression model that did not go through hyperparameter tuning. The random_state will be set to 42 for reproducibility and parameters solver='lbfgs' and max_iter=5000 to prevent any ConvergenceWarning lbfgs issue. 


We will then evaluate it cross_val_score and log loss metrics, to compute the average log loss through cross-validation on our training data, X_train and y_train. When using cross_val_score, we need to utilize "negative log loss" due to the internal implementation requirements. Consequently, the code negates the result to ensure proper computation.

In [61]:
# Importing the relevant class and function
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score


In [62]:
# Instantiating a LogisticRegression with random_state=42 for reproducibility
# The parameters solver='lbfgs' and max_iter=5000 could prevent any ConvergenceWarning lbfgs issue but are intentionally 
# not indicated to keep a vanilla model, despite ConvergenceWarnings .
# baseline_model = LogisticRegression(random_state=42, solver='lbfgs', max_iter=1000)
baseline_model = LogisticRegression(random_state=42)

In [63]:
# Using cross_val_score with scoring="neg_log_loss" to evaluate the model on X_train and y_train
baseline_neg_log_loss_cv = cross_val_score(baseline_model, X_train, y_train, scoring="neg_log_loss")

baseline_neg_log_loss_cv = -(baseline_neg_log_loss_cv.mean())
baseline_neg_log_loss_cv

C:\Users\albane.colmenares\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\albane.colmenares\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

0.5551881222011265

In [64]:
# Calculating log loss if we had a model that just chose the majority class: 0 every time 

from sklearn.metrics import log_loss
log_loss(y_train, np.zeros(len(y_train)))

19.574414143575762

Because loss is a metric where lower is better, our baseline model in an improvement over simply guessing the majority class every time. 

Although it is difficult to interpret the provided log loss, the value 0.555 will be a useful baseline as we continue modeling to see if we are actually making improvements or just getting slightly better performance by chance.  

#### 5. a. 3. Writing a Custom Cross Validation Function

Because we are using preprocessing techniques that differ for train and validation data, we will need a customer function rather than simply preprocessing the entire X_train and using cross_val_score from scikit-learn. 

---------------------NOTE---------------------

Check more precisely:
* where scaling is necessary 
* class imbalance after binary was applied

---------------------NOTE---------------------

---------------------REPHRASE THIS---------------------

Because of class imbalance, we should add some kind of resampling
step. Specifically we'll use SMOTE.

We are getting a ConvergenceWarning, which means that the gradient
descent algorithm within the logistic regression is failing to find
an optimized answer. We can also see from looking at the dataset that
some of our variables are quite small (0 or 1) while others range in
the thousands. This indicates that we should add scaling to help
"flatten" the landscape being iterated over by normalizing the
various units of the different columns.


SMOTE = synthetic minority oversampling. It creates synthetic examples of the minority class to help train the model.
SMOTE is not a scikit-learn transformer, as opposed to StandardScaler. After SMOTE was fit on the training data, the transformer should not be applied on the training <i>and</i> test data. If we were, we would be generating metrics based on synthetic data, not actual one.


---------------------REPHRASE THIS---------------------

StandardScaler, on the other hand is a transformer so it is used to transform both the train and test data from the fit from the training data only.  
Fit transform should be done after having performed cross validation to prevent leakage - not straight on X_train. 

StandardScaler, on the other hand is a transformer so it is used to transform both the train and test data from the fit from the training data only.  
Fit transform should be done after having performed cross validation to prevent leakage - not straight on X_train. 

In [65]:
# Equivalent of the code above, using StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

# Negative log loss doesn't exist as something we can import,
# but we can create it
neg_log_loss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# Instantiate the model (same as previous example)
baseline_model = LogisticRegression(random_state=42)

# Create a list to hold the score from each fold
kfold_scores = np.ndarray(5)

# Instantiate a splitter object and loop over its result
kfold = StratifiedKFold()
for fold, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):
    # Extract train and validation subsets using the provided indices
    X_t, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_t, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Clone the provided model and fit it on the train subset
    temp_model = clone(baseline_model)
    temp_model.fit(X_t, y_t)
    
    # Evaluate the provided model on the validation subset
    neg_log_loss_score = neg_log_loss(temp_model, X_val, y_val)
    kfold_scores[fold] = neg_log_loss_score
    
-(kfold_scores.mean())

C:\Users\albane.colmenares\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\albane.colmenares\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mod

0.5551881222011265

When you set sampling_strategy=0.9, SMOTE will generate synthetic samples for the minority class to make it larger. It will aim to create enough synthetic samples so that the minority class has 90% as many samples as the majority class.

In [85]:
# Importing relevant sklearn and imblearn classes
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

def custom_cross_val_score(estimator, X, y):
    # Creating a list to hold the scores from each fold
    kfold_train_scores = np.ndarray(5)
    kfold_val_scores = np.ndarray(5)

    # Instantiating a splitter object and loop over its result
    kfold = StratifiedKFold(n_splits=5)
    for fold, (train_index, val_index) in enumerate(kfold.split(X, y)):
        # Extracting train and validation subsets using the provided indices
        X_t, X_val = X.iloc[train_index], X.iloc[val_index]
        y_t, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Instantiating StandardScaler
        scaler = StandardScaler()
        # Fitting and transforming X_t
        X_t_scaled = scaler.fit_transform(X_t)
        # Transforming X_val
        X_val_scaled = scaler.transform(X_val)
        
        # Instantiating SMOTE with random_state=42 and sampling_strategy=0.85
        sm = SMOTE(random_state=42, sampling_strategy=0.85)
        # Fitting and transforming X_t_scaled and y_t using sm
        X_t_oversampled, y_t_oversampled = sm.fit_resample(X_t_scaled, y_t)
        
        # Cloning the provided model and fitting it on the train subset
        temp_model = clone(estimator)
        temp_model.fit(X_t_oversampled, y_t_oversampled)
        
        # Evaluating the provided model on the train and validation subsets
        neg_log_loss_score_train = neg_log_loss(temp_model, X_t_oversampled, y_t_oversampled)
        neg_log_loss_score_val = neg_log_loss(temp_model, X_val_scaled, y_val)
        kfold_train_scores[fold] = neg_log_loss_score_train
        kfold_val_scores[fold] = neg_log_loss_score_val
        
    return kfold_train_scores, kfold_val_scores

# Running LogisticRegression with the parameters class_weight={1: 0.85} which align with the weighting created by our SMOTE process
model_with_preprocessing = LogisticRegression(random_state=42, class_weight={1: 0.85})


preprocessed_train_scores, preprocessed_neg_log_loss_cv = custom_cross_val_score(model_with_preprocessing, X_train, y_train)
- (preprocessed_neg_log_loss_cv.mean())

0.5437811307318701

Here's what each of these components means:

**StratifiedKFold**: Stratified K-Fold cross-validation is a technique that ensures each fold of your data maintains the same class distribution as the whole dataset. This is particularly important when dealing with imbalanced datasets. By using StratifiedKFold, you're getting a more robust estimate of your model's performance.

**StandardScaler**: StandardScaler is a method for scaling your features (independent variables) so that they have a mean of 0 and a standard deviation of 1. This is often used in machine learning to ensure that different features are on a similar scale, which can help improve the performance of some algorithms, including logistic regression.

**SMOTE (Synthetic Minority Over-sampling Technique)**: SMOTE is an oversampling technique used to address class imbalance in a dataset. It generates synthetic samples for the minority class by interpolating between existing samples. In your case, you're using SMOTE with a sampling ratio of 0.85, which means it's creating synthetic samples to increase the number of instances in the minority class.

Now, let's talk about the negative log loss values:

**Average neg_log_loss of 0.5437811307318701** with the configured pipeline: This indicates the performance of your model when using the combination of StratifiedKFold, StandardScaler, and SMOTE with a sampling ratio of 0.85. A lower negative log loss is better, so a value of 0.5437811307318701 suggests that your model is reasonably good at predicting probabilities for the target classes.

**Neg_log_loss of 0.5551881222011265** for vanilla Logistic Regression: This indicates the performance of a basic Logistic Regression model without the additional preprocessing and sampling techniques. A higher negative log loss in this case suggests that the vanilla model is not performing as well as the configured pipeline. This could be due to issues such as class imbalance, feature scaling, or the absence of synthetic samples for the minority class.

In summary, the results suggest that your configured pipeline, which includes StratifiedKFold, StandardScaler, and SMOTE, is producing a model with better performance (lower negative log loss) compared to a basic Logistic Regression model without these enhancements. This improvement is likely due to the combination of better cross-validation, feature scaling, and addressing class imbalance using SMOTE.

#### 5. a. 4. Building and Evaluating Additional Logistic Regression Models

---------------------------re-read---------------------------

Using the function created in the previous step, build multiple logistic regression models with different hyperparameters in order to minimize log loss

---------------------------re-read---------------------------

Next, we want to verify whether our model is overfitting, meaning have we over-trained on the training data and the model cannot perform well on the test data. 
We will then examine and compare the training and validation scores from our existing modeling process

In [86]:
# Comparing scores 
print("Train:     ", -preprocessed_train_scores)
print("Validation:", -preprocessed_neg_log_loss_cv)

Train:      [0.54181661 0.54190109 0.5425973  0.54081018 0.54401658]
Validation: [0.54682556 0.54477032 0.54206622 0.54856481 0.53667874]


In [87]:
print("""
While SMOTE makes it somewhat challenging to compare these numbers directly,
it does not appear that we are overfitting. Overfitting would mean getting
significantly better scores on the training data than the validation data

We are getting better metrics on the validation data (where synthetic
examples have not been added) so it does not appear that we are overfitting

""")


While SMOTE makes it somewhat challenging to compare these numbers directly,
it does not appear that we are overfitting. Overfitting would mean getting
significantly better scores on the training data than the validation data

We are getting better metrics on the validation data (where synthetic
examples have not been added) so it does not appear that we are overfitting




#### Verifying if under-fitting

In [88]:
model_with_preprocessing.get_params()

{'C': 1.0,
 'class_weight': {1: 0.85},
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

That first key-value pair, 'C': 1.0, specifies the regularization strength. 

---------Rephrase------------
Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
---------Rephrase------------

#### Reducing Regularization

In [89]:
model_less_regularization = LogisticRegression(
    random_state=42,
    class_weight={1: 0.28},
    C=1e5
)

In [91]:
# Now, evaluating that model using custom_cross_val_score
# Replace None with appropriate code
less_regularization_train_scores, less_regularization_val_scores = custom_cross_val_score(
    model_less_regularization,
    X_train,
    y_train
)


print("Previous Model")
print("Train average:     ", -preprocessed_train_scores.mean())
print("Validation average:", -preprocessed_neg_log_loss_cv.mean())
print("Current Model")
print("Train average:     ", -less_regularization_train_scores.mean())
print("Validation average:", -less_regularization_val_scores.mean())

Previous Model
Train average:      0.5422283532859743
Validation average: 0.5437811307318701
Current Model
Train average:      0.693720452171832
Validation average: 0.6974181024411884


#### 5. a. 5. Choosing and Evaluating a Final Model

Preprocess the full training set and test set appropriately, then evaluate the final model with various classification metrics in addition to log loss. 

## 6. Evaluation

Which model best meets the business objectives?

After you finish refining your models, you should provide 1-3 paragraphs in the notebook discussing your final model.

Choosing the right **classification metrics**

## 7. Findings & Recommendations

**Predictive** approach

A predictive finding might include:

* How well your model is able to predict the target
* What features are most important to your model


A predictive recommendation might include:

* The contexts/situations where the predictions made by your model would and would not be useful for your stakeholder and business problem
* Suggestions for how the business might modify certain input variables to achieve certain target results

## 8. Limits & Next Steps

## \*\*Appendix \*\*